In [5]:
import re
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import bs4
import ssl
import urllib3
import pandas as pd
import faiss
import os
from langchain.embeddings import HuggingFaceEmbeddings
import torch
import gc

def instruct_structure(prompt):
    input_text, output_text = prompt.split('### target')
    input_text = input_text.replace('### glossaries', '### glossary').replace('\n* ', '\n• ')
    input_text = re.sub(r"\[[^\]]+\] ", "[UNK] ", input_text)
    return input_text

project_id = "prod-ai-project"

from google.cloud import bigquery
client = bigquery.Client(project=project_id)
sql = """select series_id, episode_id, org_input_text, org_output_text, prompt 
        from webtoon_translation.structured_240820_ep_line
        where data_split = 'romance_valid'"""
df = client.query(sql).result().to_dataframe()
from tqdm import tqdm
tqdm.pandas()
df['prompt'] = df['prompt'].progress_apply(lambda x: instruct_structure(x))

/home/bun.2/.cache/pypoetry/virtualenvs/poetry-env-eDEwtiIl-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<00:00, 18410.26it/s]


In [6]:
def substring_containment(s1, s2, penta_weight = 5, quad_weight=4, tri_weight=3, bi_weight=2, char_weight=1):
    def get_ngrams(s, n):
        s = ''.join(s.split()) #space 제거  
        return {s[i:i+n] for i in range(len(s) - (n-1))}

    s1_char = get_ngrams(s1, 1)
    s2_char = get_ngrams(s2, 1)
    
    s1_bigrams = get_ngrams(s1, 2)
    s2_bigrams = get_ngrams(s2, 2)
    
    s1_trigrams = get_ngrams(s1, 3)
    s2_trigrams = get_ngrams(s2, 3)

    s1_quadgrams = get_ngrams(s1, 4)
    s2_quadgrams = get_ngrams(s2, 4)

    s1_pentagrams = get_ngrams(s1, 5)
    s2_pentagrams = get_ngrams(s2, 5)

    #char 계산
    common_char = s1_char & s2_char
    char_score = len(common_char)
    
    # bi-gram 포함도 계산
    common_bigrams = s1_bigrams & s2_bigrams
    bigram_score = len(common_bigrams)

    # tri-gram 포함도 계산
    common_trigrams = s1_trigrams & s2_trigrams
    trigram_score = len(common_trigrams)

    # quad-gram 포함도 계산
    common_quadgrams = s1_quadgrams & s2_quadgrams
    quadgram_score = len(common_quadgrams)

    # penta-gram 포함도 계산
    common_pentagrams = s1_pentagrams & s2_pentagrams
    pentagram_score = len(common_pentagrams)

    # 최종 점수 계산 
    # s1입장에서 점수를 한 번 구하고, s2 입장에서 점수를 한 번 구해서 둘을 평균
    denominator = char_weight + bi_weight + tri_weight + quad_weight + penta_weight
    numerator = char_weight * char_score + \
                 bi_weight * bigram_score + \
                 tri_weight * trigram_score + \
                 quad_weight * quadgram_score + \
                 penta_weight * pentagram_score
    final_score = numerator / denominator
    
    return final_score

def adjusted_edit_distance(s1, s2):
    """Levenshtein 거리 기반 + 공통 단어 + 포함도 반영"""
    containment_ratio = substring_containment(s1, s2)

    # 겹치는 문자가 없으면 후보 제외
    if containment_ratio == 0:
        return -float('inf')
        
    adjusted_distance = containment_ratio
    return adjusted_distance

def find_most_similar_sentence(target, sentences):
    """주어진 문장 목록에서 target 문장과 가장 유사한 문장을 찾음"""
    similarities = []
    for sentence in sentences:
        #길이 3이내로 차이나는 것만 담기
        if len(target) >= len(sentence) + 3:
            #print(len(sentence), len(target))
            similarities.append((sentence, adjusted_edit_distance(target, sentence)))
    similarities.sort(key=lambda x: x[1],reverse=True)  # 거리 기준 정렬 (높을수록 유사)
    return similarities[:3]

In [27]:
def extract_top(question):
    file_path = "./data/idiom_dict.txt"
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    origin_lines = [line.split('>')[0].strip() for line in lines]

    result = find_most_similar_sentence(question, origin_lines)
    #문법적 유사성 판단 결과 
    #1점 이하는 문법적 유사성이 낮다고 판단
    SYNTATIC_THRESHOLD = 1.0
    result = [r[0] for r in result if r[1] > SYNTATIC_THRESHOLD]
    #문법적 유사성으로 1차 필터링
    print('문장 : ',question)
    print('문법적으로 유사한 후보군들 : ',result[:3])
    if len(result) == 0:
        return None

    #의미적 유사성 비교
    docs = [Document(page_content=f"{line}") for line in result]
    embeddings = HuggingFaceEmbeddings(model_name="nlpai-lab/KURE-v1")
    #embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

    vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)
    retriever = vectorstore.as_retriever(search_type='similarity_score_threshold',
                                         search_kwargs={'k':5,'score_threshold' : 0.3})    
    
    docs = retriever.invoke(question) 
    
    if hasattr(vectorstore, "index") and isinstance(vectorstore.index, faiss.Index):
        # FAISS 인덱스를 CPU로 변환 (GPU 해제)
        vectorstore.index = faiss.index_gpu_to_cpu(vectorstore.index)
    
    # FAISS 객체 삭제
    del vectorstore
    # Python 가비지 컬렉션 실행
    gc.collect()
    # PyTorch GPU 캐시 정리
    torch.cuda.empty_cache()

    #2차 필터링
    if docs:
        print(docs)
        return docs[0].page_content
    else:
        return None
   

In [28]:
data_idx = 5
data = df['prompt'][data_idx]
example = data.split("### source")[1].strip()
#print(example)

In [29]:
print(find_most_similar_sentence("왜 남주가 벌써 내 코앞에서 굴러다니고 있는 거지?",["코 앞이다"]))
print(find_most_similar_sentence("한가지 억울한 점이 있다면",["싹이 있다"]))

[('코 앞이다', 0.3333333333333333)]
[('싹이 있다', 0.6666666666666666)]


In [30]:
print(substring_containment("왜 남주가 벌써 내 코앞에서 굴러다니고 있는 거지?", "코 앞이다"))

0.3333333333333333


In [31]:
print(extract_top("그를 눈엣가시처럼 여겨 죽이고 싶어 하는 자들은 많았다."))

문장 :  그를 눈엣가시처럼 여겨 죽이고 싶어 하는 자들은 많았다.
문법적으로 유사한 후보군들 :  ['눈엣가시처럼 여기다', '눈엣가시다', '눈엣가시 같다']
[Document(id='bf173b44-0033-4e61-99e4-8c5987258028', metadata={}, page_content='눈엣가시처럼 여기다'), Document(id='20bddeeb-14e7-4550-8c8f-746801553a94', metadata={}, page_content='눈엣가시 같다')]
눈엣가시처럼 여기다


In [33]:
sens = example.split('\n')

hints = []
for sen in sens:
    q = sen.split('[UNK]')[1].strip()
    result = extract_top(q)
    if result:
        idiom = result
        print(idiom)
        print()
        hints.append(idiom)
    else:
        print()

문장 :  생각보다 좀 걸리네.
문법적으로 유사한 후보군들 :  []

문장 :  붕대 가는 건 별로 안 걸릴 것 같았는데.
문법적으로 유사한 후보군들 :  []

문장 :  ‘세상 물정 모르는 아가씨’라고 말하는 눈
문법적으로 유사한 후보군들 :  ['세상 물정 모르다', '세상 물정 모른다', '세상물정을 모르다']
[Document(id='d04dc3b8-2a04-4e89-865f-80c325a558ee', metadata={}, page_content='세상 물정 모르다'), Document(id='eaf2ea36-83c2-4551-af38-dc87ba35ba36', metadata={}, page_content='세상 물정 모른다'), Document(id='c3b10421-e2f9-47f0-a7a3-051e07f91f35', metadata={}, page_content='세상물정을 모르다')]
세상 물정 모르다

문장 :  아까 말하려다 못한 이야기를 하려고 했건만.
문법적으로 유사한 후보군들 :  ['허무맹랑한 이야기를 하다', '이야기를 꺼내다', '이야기를 풀어놓다']
[Document(id='232e8e87-4930-4082-8a0a-345cf6544d28', metadata={}, page_content='이야기를 꺼내다'), Document(id='8c14dc94-a406-41d9-8ceb-4fb6940b2001', metadata={}, page_content='허무맹랑한 이야기를 하다'), Document(id='83633cab-8d58-4cd3-843d-de3fd6e92d53', metadata={}, page_content='이야기를 풀어놓다')]
이야기를 꺼내다

문장 :  나도 나름 열심히 일을 하고 있다구.
문법적으로 유사한 후보군들 :  ['등이 휠 정도로 열심히 일하다', '허리가 휠 정도로 열심히 일하다', '개미처럼 열심히 일하다']
[Document(id='ac96bdcf-d273-4661-b121-f970fd81e

No relevant docs were retrieved using the relevance score threshold 0.3



문장 :  그래서 통 감이 안 잡힌단 말이야.
문법적으로 유사한 후보군들 :  ['감이 안 잡히다']
[Document(id='7a97f497-3dce-48f5-b2f0-00df5d241495', metadata={}, page_content='감이 안 잡히다')]
감이 안 잡히다

문장 :  애초에 소설은 여주인 유니스의 시점으로 진행 되었고…
문법적으로 유사한 후보군들 :  ['일사천리로 진행되다']


No relevant docs were retrieved using the relevance score threshold 0.3



문장 :  그 이야기 속에서 시그렌에게 부여된 역할은 간단했다.
문법적으로 유사한 후보군들 :  []

문장 :  잘생기고, 헌신적이고, 다정하고.
문법적으로 유사한 후보군들 :  []

문장 :  오직 유니스에게만 사랑을 주는 완벽한 연인.
문법적으로 유사한 후보군들 :  []

문장 :  누구나 한 번쯤은 비현실적인 사랑을 받는걸 꿈꾸지 않는가?
문법적으로 유사한 후보군들 :  ['황금을 주어도 바꾸지 않는다', '사랑을 받다', '사랑을 받고 자라다']
[Document(id='8c9910a9-3238-4eaf-ba76-ef1a0762c483', metadata={}, page_content='사랑을 받다')]
사랑을 받다

문장 :  항상 날 신뢰하고, 내 편으로 있어 줄 사람을.
문법적으로 유사한 후보군들 :  []

문장 :  그런 시그렌의 성격만 상상했으니까,
문법적으로 유사한 후보군들 :  []

문장 :  지금 어떻게 다뤄야 할지 모르겠어.
문법적으로 유사한 후보군들 :  []

문장 :  의도가 살짝 불순해서 그런가.
문법적으로 유사한 후보군들 :  []

문장 :  난 미안함과 애정, 계산적인 마음이 섞여서 가까워지고 싶은 건데.
문법적으로 유사한 후보군들 :  []

문장 :  그는 그걸 이유 없는 호의로 여겨 경계하는 모양이다.
문법적으로 유사한 후보군들 :  []

문장 :  섣불리 다가갔다가 날 더 싫어하게 되면 미래가 불안해지는데.
문법적으로 유사한 후보군들 :  []

문장 :  아가씨.
문법적으로 유사한 후보군들 :  []

문장 :  끝났나요?
문법적으로 유사한 후보군들 :  []

문장 :  그렇긴 합니다만….
문법적으로 유사한 후보군들 :  []

문장 :  문제가 생겼나요?
문법적으로 유사한 후보군들 :  []

문장 :  아뇨, 부상의 회복은 무척 순조롭습니다.
문법적으로 유사한 후보군들 :  []

문장 :  다만…….
문법적으로 유사한 후보군들 :  []

문장 :  무슨 일이 생긴 건가요?
문법적으로 유사한 후보

No relevant docs were retrieved using the relevance score threshold 0.3



문장 :  뭐, 뭐야 또….
문법적으로 유사한 후보군들 :  []

문장 :  ...어.
문법적으로 유사한 후보군들 :  []

문장 :  다친 데 없어. 그냥….
문법적으로 유사한 후보군들 :  []

문장 :  시그렌의 가슴, 팔, 옆구리를 가득 채운 흔적과 흉터.
문법적으로 유사한 후보군들 :  ['옆구리를 찌르다', '구십 구 리를 가고 한 리를 못 간다']
[Document(id='1d50353c-6926-4d7a-a425-95d40395b3ca', metadata={}, page_content='옆구리를 찌르다')]
옆구리를 찌르다

문장 :  그리고 푸른 멍들.
문법적으로 유사한 후보군들 :  []

문장 :  단순히 부상으로 남은 것들이 아니다.
문법적으로 유사한 후보군들 :  ['볕 좋은 날만 있는 것이 아니다', '하루아침에 이루어진 것이 아니다', '다 같은 나무의 잎이 아니다']
[Document(id='a813d8bb-2cc3-4cda-95a1-d5af97728896', metadata={}, page_content='하루아침에 이루어진 것이 아니다'), Document(id='bc3f0a1e-fb9b-4520-96b4-e722e7397bc1', metadata={}, page_content='다 같은 나무의 잎이 아니다'), Document(id='a40e07e5-f45c-42f2-a64b-375ca31a4fd3', metadata={}, page_content='볕 좋은 날만 있는 것이 아니다')]
하루아침에 이루어진 것이 아니다

문장 :  고의로 불에 지져지고,
문법적으로 유사한 후보군들 :  []

문장 :  일방적인 폭력에 시달리면서 생긴 흔적들이다.
문법적으로 유사한 후보군들 :  ['생활고에 시달리다']
[Document(id='a2bc3e37-7970-4bb7-ac04-dc7e4b47665e', metadata={}, page_content='생활고에 시달리다')]
생활고에 시달리다

문장 :  상처에는 오랜 세월 동안

No relevant docs were retrieved using the relevance score threshold 0.3



문장 :  내가 할 수 있는 최대한의 사과일 것이다.
문법적으로 유사한 후보군들 :  []



In [34]:
print(hints)

['세상 물정 모르다', '이야기를 꺼내다', '등이 휠 정도로 열심히 일하다', '감이 안 잡히다', '사랑을 받다', '사랑도 돌아가면서 한다', '작은 입을 열다', '죽은 사람 욕하지 말라', '엉망진창이 되다', '옆구리를 찌르다', '하루아침에 이루어진 것이 아니다', '생활고에 시달리다', '제멋대로 행동하다', '말도 안 되는 소리', '세월을 보냈다', '별거 아니다', '제 정신이 아니다', '꿈에도 생각하지 못했다', '가시처럼 느껴지다', '받아들이기 힘들다']
